# Optical Flow with Lucas-Kanade

This notebook demonstrates how to use the Lucas-Kanade optical flow method to extract the breathing signal from a video.

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

subject = 'Proband16'
setting = '101_natural_lighting'

subject_frames, params = dataset.get_video_gray(subject, setting, num_frames=30 * 10)

In [ ]:
quality_level = 0.1
quality_level_rv = 0.05

In [ ]:
import respiration.roi as roi

frame1 = subject_frames[0]
subject_roi = roi.detect_chest(frame1)

In [ ]:
import matplotlib.pyplot as plt
import respiration.utils as utils

from respiration.extractor import lucas_kanade

# Get different feature points for the first frame
default_points = lucas_kanade.select_feature_points(frame1)
special_points = lucas_kanade.select_feature_points(frame1, fpn=5)
roi_points = lucas_kanade.select_feature_points(frame1, roi=subject_roi)
special_roi = lucas_kanade.select_feature_points(frame1, roi=subject_roi, fpn=5)

# Plot the first frame with the feature points
plt.imshow(frame1, cmap='gray')

# Draw the region of interest (ROI)
# roi_x, roi_y, roi_w, roi_h = subject_roi
# plt.gca().add_patch(plt.Rectangle(
#     (roi_x, roi_y), roi_w, roi_h,
#     linewidth=1, edgecolor='r', facecolor='none'))

for iny in range(default_points.shape[0]):
    plt.scatter(default_points[iny, 0, 0],
                default_points[iny, 0, 1],
                c='r', s=2.5)

# for iny in range(special_points.shape[0]):
#     plt.scatter(special_points[iny, 0, 0],
#                 special_points[iny, 0, 1],
#                 c='b', s=2.5)

# for iny in range(roi_points.shape[0]):
#     plt.scatter(roi_points[iny, 0, 0],
#                 roi_points[iny, 0, 1],
#                 c='#FFFF00', s=2.5)

# for iny in range(special_roi.shape[0]):
#     plt.scatter(special_roi[iny, 0, 0],
#                 special_roi[iny, 0, 1],
#                 c='#FF00FF', s=2.5)

figure_dir = utils.dir_path('outputs', 'figures', 'lucas_kanade', mkdir=True)
utils.savefig(plt.gcf(), figure_dir, 'feature_points')

plt.show()

In [ ]:
# Track the movement of the feature points
feature_point_movements = lucas_kanade.track_feature_point_movement(subject_frames, special_points)

# Extract the amplitudes of the feature points
raw_signal = lucas_kanade.calculate_feature_point_amplitudes(feature_point_movements)

In [ ]:
feature_point_movements.shape

In [ ]:
# Plot the raw signal
plt.figure(figsize=(20, 6))
plt.plot(raw_signal)

In [ ]:
preprocessed_unprocessed = lucas_kanade.signal_from_amplitudes(
    raw_signal,
    use_cgof=False,
)

signal_cgof = lucas_kanade.signal_from_amplitudes(
    raw_signal,
    use_cgof=True,
)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 1, figsize=(20, 10))

# Add some space between the subplots
fig.subplots_adjust(hspace=0.5)

axs[0].plot(preprocessed_unprocessed)
axs[0].set_title('Unprocessed')

axs[1].plot(signal_cgof)
axs[1].set_title('CGOF')

plt.show()

## Evaluation

In [ ]:
gt_signal = dataset.get_breathing_signal(subject, setting)

# Make sure the ground truth signal has the same length as the predicted signal
gt_signal = gt_signal[:len(signal_cgof)]

In [ ]:
import respiration.analysis as analysis

# Calculate the frequencies using the different methods for the ground truth signal
compare = analysis.SignalCompare(
    signal_cgof,
    gt_signal,
    params.fps,
    detrend_tarvainen=True,
    normalize_signal=True,
    filter_signal=True,
)

In [ ]:
fig = plt.figure(figsize=(20, 6))
plt.plot(compare.ground_truth, label='Ground Truth Signal')
plt.plot(compare.prediction, label='Prediction')
plt.legend()
plt.show()

In [ ]:
# Show the error in beats per minute (BPM) for the different methods
compare.bpm_errors()

In [ ]:
# Show the distances between the predicted and ground truth
compare.distances()